<p style="font-weight:bold;"> <span style="font-size: 36px"> Initialize Dimensions </span> </p>

In [ ]:
#!import "//ifrs17/dev/CalculationEngine"

---

In [ ]:
await Import.FromFile("Dimensions.xlsx")
    .WithType<ReportingNode>()               .WithType<AocType>()                .WithType<DeferrableAmountType>()
    .WithType<PvAmountType>()                .WithType<Scenario>()               .WithType<LiabilityType>()
    .WithType<LineOfBusiness>()              .WithType<EstimateType>()           .WithType<EconomicBasis>()
    .WithType<Currency>()                    .WithType<PnlVariableType>()        .WithType<BsVariableType>()
    .WithType<Novelty>()                     .WithType<Profitability>()          .WithType<OciType>()
    .WithType<ValuationApproach>()           .WithType<RiskDriver>()             .WithType<ProjectionConfiguration>()    
    //.WithType<Partner>()                     .WithType<CreditRiskRating>()       
    .WithType<ExchangeRate>()
    .SnapshotMode()
    .WithTarget(DataSource)
    .ExecuteAsync()

In [ ]:
await Import.FromFile("Dimensions.xlsx").WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync()

In [ ]:
await Import.FromFile("DataNodes.xlsx").WithFormat("DataNode").WithTarget(DataSource).ExecuteAsync()

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules());
Workspace.InitializeFrom(DataSource);